In [28]:
import gzip
import json
import random
import time
from nltk import word_tokenize
from tqdm import tqdm
import contractions
import nltk
import random
import csv

import gzip
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import f1_score
from sklearn.utils import shuffle

In [15]:
#nltk.download('wordnet')
#nltk.download('omw-1.4')
#nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
random.seed(42)

In [3]:
#here
def reading_in(path):
    thing=[]
    for line in gzip.open(path):
        review_data = json.loads(line)
        subthing = dict()
        for key in review_data:
            subthing[key]= review_data[key]
        thing.append(subthing)
    return thing

In [4]:
train = reading_in('../data/classification/music_reviews_train.json.gz')
dev = reading_in('../data/classification/music_reviews_dev.json.gz') # do not use!
#test = reading_in('../data/classification/music_reviews_test_masked.json.gz') #do not use!
train = shuffle(train, random_state=42)

In [5]:
train_x = [i['reviewText'] for i in train if "reviewText" in i.keys() and "sentiment" in i.keys()]
train_y =  [i['sentiment'] for i in train if "reviewText" in i.keys() and "sentiment" in i.keys()]

dev_x = [i['reviewText'] if "reviewText" in i.keys() else "" for i in dev]
dev_y = [i['sentiment'] for i in dev]

In [25]:
###split into pos and neg
train_x_pos = []
train_x_neg = []
train_y_pos = []
train_y_neg = []
for i in range(len(train_x)):
    if train_y[i] == "positive":
        train_x_pos.append(train_x[i])
        train_y_pos.append(train_y[i])
    else:
        train_x_neg.append(train_x[i])
        train_y_neg.append(train_y[i])
train_x_neg.extend(train_x_pos[0:len(train_x_neg)])
train_y_neg.extend(train_y_pos[0:len(train_y_neg)])
balanced_x_train_s = train_x_neg
balanced_y_train_s = train_y_neg

balanced_x_train, balanced_y_train = shuffle(balanced_x_train_s, balanced_y_train_s, random_state= 42)
def tokenize_corpus(corpus):
    return [word_tokenize(contractions.fix(s.lower())) for s in corpus]
balanced_x_train = tokenize_corpus(balanced_x_train)
dev_x = tokenize_corpus(dev_x)

### read in 3 synthetic

In [64]:
def read_synth(path,deli):
    with open(path) as f:
        reader = csv.reader(f,delimiter = "\n")
        data = []
        for i in reader:
            print(i[0].split(deli))
    return data
paths = ["../data/synthetic/webscrape.csv","../data/synthetic/wordnet.csv","../data/synthetic/word2vec.csv"]
deli = [",","\t","\t"]

In [65]:
synthetic = []
for x,i in enumerate(paths):
    print(i)
    synthetic.append([i,read_synth(i,deli[x])])

 'Over', "''", ')', 'does', '.', 'As', 'a', 'member', 'of', 'the', 'American', 'Psychological', 'Association', '(', 'though', 'I', 'would', 'once', 'claim', 'to', 'spoke', 'on', 'behalf', 'of', 'them', ')', ',', 'I', 'would', 'really', 'to', 'tell', 'that', 'I', 'think', 'that', 'this', 'song', 'is', 'incredibly', 'inspiring', 'and', 'indicative', 'of', 'many', 'of', 'the', 'struggles', 'that', 'persons', 'may', 'goes', 'through', 'in', 'their', 'life', '.', 'I', 'hopefully', 'that', 'it', 'can', 'help', 'those', 'in', 'their', 'journeys', 'to', 'sobriety', '(', 'or', 'recovery', 'from', 'addiction', ')', '.', 'If', 'Nothing', 'else', ',', 'the', 'ditty', 'is', 'gorgeous', '.']
['nothing', 'can', 'make', 'a', 'song', 'serenade', 'teenaged', 'girls', 'like', 'r', 'kelly', 'and', 'get', 'away', 'with', 'it', 'for', '20', 'yeas', 'wow', 'kelly', 'how', 'did', 'you', 'do', 'it', '.']
['Along', 'with', '``', 'Be', 'My', 'Lady', "''", ',', 'Baby', 'Please', 'Stay', 'is', 'another', 'FP', 'ma

In [34]:
synthetic

[['../data/synthetic/webscrape.csv', <_csv.reader at 0x7ffb7c8a55d0>],
 ['../data/synthetic/wordnet.csv', <_csv.reader at 0x7ffb7c8a5650>],
 ['../data/synthetic/word2vec.csv', <_csv.reader at 0x7ffb7c8a54d0>]]

In [ ]:
def add_together(synt_dataset_X,synt_dataset_y, samples):
    OG_X = balanced_x_train[:samples]
    OG_y = balanced_y_train[:samples]
    train_X = OG_X + synt_datset_X
    train_y = OG_y + synt_dataset_y
    train_X, train_y = shuffle(train_X, train_y, random_state= 42)
    return train_X, train_y

In [ ]:
Scrape10_X, Scrape10_y = add_together(synt_dataset_X,synt_dataset_y, 10000)
Net10_X, Net10_y = add_together(synt_dataset_X,synt_dataset_y, 10000)
Vec10_X, Vec10_y =add_together(synt_dataset_X,synt_dataset_y, 10000)
Scrape5_X, Scrape5_y = add_together(synt_dataset_X,synt_dataset_y, 5000)
Net5_X, Net5_y = add_together(synt_dataset_X,synt_dataset_y, 5000)
Vec5_X, Vec5_y = add_together(synt_dataset_X,synt_dataset_y, 5000)

### list of datasets

In [ ]:
# Training X, training y,  description
train_list = [
    [balanced_x_train[:20000], balanced_y_train[:20000], ,'original data big'],
    [balanced_x_train[:10000], balanced_y_train[:10000],'original data small'],
    [Scrape10_X, Scrape10_y ,'WebScrape 10,000'],
    [Net10_X, Net10_y, 'WordNet 10,000'], 
    [ Vec10_X, Vec10_y,'Word2Vec 10,000' ],
    [ Scrape5_X, Scrape5_y,'WebScrape 5,000'],
    [Net5_X, Net5_y, 'WordNet 5,000'], 
    [ Vec5_X, Vec5_y,'Word2Vec 5,000' ]


]

In [10]:
MNB = Pipeline([
    ('CountVectorizer',CountVectorizer(ngram_range=(1, 2))),
    ('Tfidf',TfidfTransformer()),
    ('clf', MultinomialNB(alpha=1, fit_prior=False))
])#best params given here

In [12]:
for

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b88dfe01-c7e1-473c-bcfd-798313fc6522' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>